<a href="https://colab.research.google.com/github/deenukhan/deep_learning/blob/main/4_cnn_xception_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Notebook is inspired the book Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow
# In this Notebook We Will be training the Model on Flowers dataset and will use xception pretrained model for the task
# Here we will be learning and Implementing the concept of Transfer Learning  

# Importing the Libraries
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [3]:
#Let's Download the flowers dataset for , checkout the below link for more information 
# https://www.tensorflow.org/datasets/overview#load_a_dataset

datasets, info = tfds.load('tf_flowers', split = 'train', as_supervised=True, with_info=True)

dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_classes = info.features['label'].num_classes

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [4]:
# First let's take a look on the dataset and explore the size and other features
print("Total Data Points in the Dataset : ", dataset_size)
print("Classes : ", class_names)

# Let's Checkout the shape of the few example
# Here we can see that shape of flower images are not same and as Xception Model excepts the 224 × 224 Shape
# We need to resize them before feeding into the model
for image, label in datasets.take(3):
    print(image.shape, label)

Total Data Points in the Dataset :  3670
Classes :  ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
(333, 500, 3) tf.Tensor(2, shape=(), dtype=int64)
(212, 320, 3) tf.Tensor(3, shape=(), dtype=int64)
(240, 320, 3) tf.Tensor(3, shape=(), dtype=int64)


In [5]:
# We don't have testing and validation datasets, Let's create the same 
# We will be using tfds.load api for this
# We can skip the stpe of loading the datsets in earlier cell, if we are performing below code

test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers", split = ['train[:10%]', 'train[10%:25%]', 'train[25%:]'],
    as_supervised = True
)

In [6]:
# Next we must preprocess the images. The CNN expects 224 × 224 images, 
# so we need to resize them. We also need to run the images through Xception’s preprocess_input() function

def pre_process(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label
    

In [7]:
# Now Let’s apply this preprocessing function to all three datasets, shuffle the training 
# set, and add batching and prefetching to all the datasets

batch_size = 32
train_set_raw = train_set_raw.shuffle(1000)
train_set = train_set_raw.map(pre_process).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(pre_process).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(pre_process).batch(batch_size).prefetch(1)


In [13]:
# let’s load an Xception model, pretrained on ImageNet. We exclude the top of the network by setting include_top=False

base_model = keras.applications.xception.Xception(weights='imagenet', include_top=False)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(avg)
model = keras.Model(inputs = base_model.input, outputs = output)

In [14]:
# Let's Freeeze the all the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# We could have directly freeze all the layers by below code
# base_model.trainable = False

In [15]:
# Now Let's compile the model and start the training 
from datetime import datetime
print("Model Training Started with Colab GPU at ", datetime.now())
model.compile(optimizer=keras.optimizers.SGD(lr = 0.2, momentum= 0.9, decay = 0.01),
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])
model.fit(train_set, epochs=5, validation_data=valid_set)
print("Model Training Finished with Colab GPU at ", datetime.now())


Model Training Started with Colab GPU at  2021-04-26 14:16:20.505811
Epoch 1/5
86/86 [==============================] - 16s 167ms/step - loss: 1.5566 - accuracy: 0.7004 - val_loss: 0.9347 - val_accuracy: 0.8494
Epoch 2/5
86/86 [==============================] - 15s 169ms/step - loss: 0.4657 - accuracy: 0.9054 - val_loss: 0.7178 - val_accuracy: 0.8603
Epoch 3/5
86/86 [==============================] - 15s 174ms/step - loss: 0.2211 - accuracy: 0.9466 - val_loss: 0.7075 - val_accuracy: 0.8621
Epoch 4/5
86/86 [==============================] - 15s 169ms/step - loss: 0.1229 - accuracy: 0.9595 - val_loss: 0.6834 - val_accuracy: 0.8693
Epoch 5/5
86/86 [==============================] - 14s 164ms/step - loss: 0.0566 - accuracy: 0.9809 - val_loss: 0.6746 - val_accuracy: 0.8784
Model Training Finished with Colab GPU at  2021-04-26 14:17:35.141469


In [16]:
# After training the model for a few epochs, its validation accuracy should reach about 
# 75–80% and stop making much progress. This means that the top layers are now pretty well trained, 
# so we are ready to unfreeze all the layers (or you could try unfreezing just the top ones) and continue 
# training (don’t forget to compile the model when you freeze or unfreeze layers). 
# This time we use a much lower learning rate to avoid damaging the pretrained weights

for layer in base_model.layers:
    layer.trainable = True

print("Model Training Started with Colab GPU at ", datetime.now())
model.compile(optimizer=keras.optimizers.SGD(lr = 0.01, momentum= 0.9, decay = 0.001),
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])
model.fit(train_set, epochs=5, validation_data=valid_set)
print("Model Training Finished with Colab GPU at ", datetime.now())

# Here We can observe that Validation accuracy has increased than our earlier model
# And as In this model we are training all the layers, it took more time

Model Training Started with Colab GPU at  2021-04-26 14:17:48.635188
Epoch 1/5
86/86 [==============================] - 60s 661ms/step - loss: 0.4657 - accuracy: 0.8523 - val_loss: 0.4448 - val_accuracy: 0.8675
Epoch 2/5
86/86 [==============================] - 55s 640ms/step - loss: 0.0772 - accuracy: 0.9776 - val_loss: 0.3472 - val_accuracy: 0.8984
Epoch 3/5
86/86 [==============================] - 56s 647ms/step - loss: 0.0329 - accuracy: 0.9923 - val_loss: 0.3025 - val_accuracy: 0.9147
Epoch 4/5
86/86 [==============================] - 56s 646ms/step - loss: 0.0062 - accuracy: 0.9982 - val_loss: 0.3265 - val_accuracy: 0.9220
Epoch 5/5
86/86 [==============================] - 56s 646ms/step - loss: 0.0076 - accuracy: 0.9979 - val_loss: 0.3180 - val_accuracy: 0.9347
Model Training Finished with Colab GPU at  2021-04-26 14:22:30.515046
